In [69]:
import os
import time
import random
import numpy as np
import pandas as pd
from fastprogress import master_bar, progress_bar
from easydict import EasyDict as edict

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms, models
from  torch.utils.data import DataLoader, Dataset
import torch.optim.lr_scheduler as lr_scheduler
# from CenterLoss import CenterLoss
from torch.autograd.function import Function
from torchsummary import summary

from sklearn.model_selection import KFold
from sklearn.metrics import recall_score

%matplotlib inline
import matplotlib.pyplot as plt

In [38]:
SEED = 2020
EPOCH = 50
IMG_SIZE = 28
NUM_CLASSES = 10
BATCH_SIZE = 128

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [39]:
def seed_torch(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    
seed_torch(SEED)

In [40]:
class MnistDataset(Dataset):
    
    def __init__(self, df, n_channels=1, is_train=True, transforms=None):
        self.data = df.iloc[:, 1:].values
#         self.fnames = df['image_id'].values
        self.n_channels = n_channels
        self.labels = df['label'].values
        self.transforms = transforms
        self.is_train = is_train
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        image = self.data[idx, :].reshape(IMG_SIZE, IMG_SIZE).astype(np.uint8)
        image = (image*(255.0/image.max())).astype(np.uint8)
        
        if self.transforms:
            if self.transforms.albumentations:
                aug = Augmentation().get_augmentation(self.transforms.albumentations)
                augmented = aug(image=image)
                image = augmented['image'].astype(np.float32)

        image = image.reshape(1, IMG_SIZE, IMG_SIZE).astype(np.float32)
        if self.n_channels > 1:
            image = np.concatenate([image for i in range(self.n_channels)], axis=0)

        if self.is_train:
            label = self.labels[idx]
            return image, label
        else:
            return image

In [97]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter
import math

from models.resnet import resnet50


# Source: https://github.com/ronghuaiyang/arcface-pytorch/blob/master/models/metrics.py
class ArcMarginProduct(nn.Module):
    r"""Implement of large margin arc distance: :
        Args:
            in_features: size of each input sample
            out_features: size of each output sample
            s: norm of input feature
            m: margin
            cos(theta + m)
        """
    def __init__(self, in_features, out_features, s=30.0, m=0.50, easy_margin=False):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.weight = Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt((1.0 - torch.pow(cosine, 2)).clamp(0, 1))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------------
        # one_hot = torch.zeros(cosine.size(), requires_grad=True, device='cuda')
        one_hot = torch.zeros(cosine.size(), device='cuda')
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)  # you can use torch.where if your torch.__version__ is 0.4
        output *= self.s
        # print(output)

        return output


class ArcFace(nn.Module):
    def __init__(self, model, s=30.0, m=0.50, easy_margin=False):
        super(ArcFace, self).__init__()
        self.model = nn.Sequential(*list(model.modules())[:-1])
        self.in_features = getattr(list(model.modules())[-1], 'in_features')
        self.arcface = ArcMarginProduct(self.in_features, 1000, s=s, m=m, easy_margin=easy_margin)

    def forward(self, x):
        x = self.model(x)
        x = self.arcface(x)
        return x


def arcface_resnet50(pretrained=False):
    base_model = resnet50(pretrained)
    return ArcFace(base_model, s=30.0, m=0.50, easy_margin=False)

In [118]:
cfg = {
    's': 30.0,
    'm': 0.50,
    'easy_margin': False
}

In [119]:
a = ArcMarginProduct(2048, 10, **cfg)

In [98]:
# train_df = pd.read_csv('../data/input/train.csv')

In [114]:
model = models.resnet18()
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [115]:
fc_name, fc_layer = list(model.named_children())[-1]
in_features = getattr(fc_layer, 'in_features')
setattr(getattr(model, fc_name), 'out_features', in_features)

In [111]:
fc_name, fc_layer = list(model.named_children())[-1]

In [112]:
fc_name

'fc'

In [113]:
fc_layer

Linear(in_features=512, out_features=1000, bias=True)

In [117]:
summary(model.cuda(), (3, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 14, 14]           9,408
       BatchNorm2d-2           [-1, 64, 14, 14]             128
              ReLU-3           [-1, 64, 14, 14]               0
         MaxPool2d-4             [-1, 64, 7, 7]               0
            Conv2d-5             [-1, 64, 7, 7]          36,864
       BatchNorm2d-6             [-1, 64, 7, 7]             128
              ReLU-7             [-1, 64, 7, 7]               0
            Conv2d-8             [-1, 64, 7, 7]          36,864
       BatchNorm2d-9             [-1, 64, 7, 7]             128
             ReLU-10             [-1, 64, 7, 7]               0
       BasicBlock-11             [-1, 64, 7, 7]               0
           Conv2d-12             [-1, 64, 7, 7]          36,864
      BatchNorm2d-13             [-1, 64, 7, 7]             128
             ReLU-14             [-1, 6

In [ ]:
class ArcFace()